<a href="https://colab.research.google.com/github/Samuel-K95/iCog-PC-Task/blob/main/PC_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step by step breakdown of Predictive Coding.

1. Input Clamping

    - Fix the input layer to the data (e.g., an image or signal).
    - This is the “sensory input” that higher layers must explain.

2. Initialize Latent States

    - Randomly initialize hidden variables
    - These represent initial “guesses” about hidden causes of the input.

3. Top-Down Prediction

    - Each higher layer generates predictions for the layer below

4. Prediction Error Computation

    - For every layer, compute the mismatch:
    - These errors measure surprise or “unexplained” input.

5. Bottom-Up Error Propagation

    - Errors flow upwards in the hierarchy.

    - Lower-layer errors inform higher layers about how their predictions failed.

6. Inference: Update Latent States

    - Latent states are updated iteratively to reduce total error
    - This fast loop continues until the system “settles” (converges).

7. Recompute Predictions & Errors

    - After each latent update, recompute predictions and errors.

    - Repeat steps (3–6) for several iterations.

    - This ensures the network stabilizes into a consistent explanation of the input.

8. Weight Update (Learning)

    - Once inference converges, update generative weights
    - Local Hebbian-like rule: depends only on presynaptic activity and postsynaptic error.

9. Repeat for New Inputs

    - Restart the cycle with the next data sample (or minibatch).

    - Over time, weights adapt so predictions improve, reducing average error.